In [45]:
from road_segments import *
from os import listdir
from os.path import isfile, join

%load_ext autoreload
%autoreload 2


reloaded
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
request_url = build_url(gps_raw[:100])
result = requests.get(request_url, params=params)
response = result.json()

street_times = get_street_times(response)

street_points = get_street_points(street_times)
street_frame_bounds = get_street_bounds(street_times)

method      = o3d.registration.GlobalOptimizationLevenbergMarquardt()
criteria    = o3d.registration.GlobalOptimizationConvergenceCriteria()
option      = o3d.registration.GlobalOptimizationOption(
            max_correspondence_distance=0.05,
            edge_prune_threshold=0.25,
            preference_loop_closure=0.01,
            reference_node=0)


pg = o3d.io.read_pose_graph(dataset + "/pose/full.json")

g2n = get_gps_to_node(pg, gps_raw, 0)

#fullpg_to_segment['street'][segment][index] = original
fullpg_idxmap = {}

street_g2pg = {}
import os
for street, bounds in street_frame_bounds.items():
    print("Working on {}".format(street))
    
    npgs, g2pg, idxmap = chunk_pg(pg, g2n, int(bounds['start']), int(bounds['stop']))
    
    street_g2pg[street] = g2pg
    fullpg_idxmap[street] = idxmap
    print("Got {} pose graphs".format(len(npgs)))
    
    
    for idx in range(len(npgs)):
        streetdir = dataset + "/pose/" + street.replace(" ", "_")
        
        if not os.path.isdir(streetdir):
            os.mkdir(streetdir)
            
        o3d.registration.global_optimization(road, method, criteria, option)

        o3d.io.write_pose_graph(streetdir + "/{0:03}.json".format(idx), npgs[idx])
    


Working on Rigi Avenue
Got 13 pose graphs
Working on Coughlin Avenue
Got 3 pose graphs
Working on Colby Avenue
Got 11 pose graphs
Working on Tyson Place
Got 3 pose graphs


In [82]:

method      = o3d.registration.GlobalOptimizationLevenbergMarquardt()
criteria    = o3d.registration.GlobalOptimizationConvergenceCriteria()
option      = o3d.registration.GlobalOptimizationOption(
            max_correspondence_distance=0.05,
            edge_prune_threshold=0.25,
            preference_loop_closure=0.01,
            reference_node=0)

street = 'Rigi Avenue'

streetdir = dataset + "/pose/" + street.replace(" ", "_")


for fidx in range(len(street_g2pg[street])):
        
    if fidx>5:
        break
        
    print("reading: {}".format(fidx))
    road = o3d.io.read_pose_graph(streetdir + "/{0:03}.json".format(fidx))
        
    sp = street_g2pg[street][fidx]
    
    gps_start = np.array([sp[0][4],sp[0][5],1])
    gps_stop = np.array([sp[-1][4],sp[-1][5],1])
    gps_bearing = gps_stop - gps_start

    print(gps_bearing)
    
    road_start = road.nodes[0].pose[:3,3]
    road_stop = road.nodes[len(road.nodes)-1].pose[:3,3]
    road_bearing = road_start - road_stop
    
    print(road_bearing)
    
    #camera travels in Z direction, need to rotate it to align with gps bearing
    t = R.from_dcm(get_rotation(gps_bearing, road_bearing)).inv()
    
    print(t.apply(gps_bearing))

    #leftover disparity is dominated by pitch of camera 
    #camera_pitch = zrot.apply(gps_bearing)
    #pitch_rot = R.from_dcm(get_rotation(camera_pitch, road_bearing))

    #t = R.from_dcm(np.identity(3))
    
    gls = lineset_from_gps(sp, R.from_dcm(np.identity(3)))
    pls = lineset_from_nodes(road)
    
    i = np.identity(4)
    i[:3,:3] = t.as_dcm()
    
    gls.transform(i)
    o3d.visualization.draw_geometries([gls, pls])
    
    
    
#     inject_gps_edges(road, sp, fullpg_idxmap[street][fidx][0], t)
#     o3d.io.write_pose_graph(streetdir + "/{0:03}_test.json".format(fidx), road)
    
#     o3d.registration.global_optimization(road, method, criteria, option)
#     o3d.io.write_pose_graph(streetdir + "/{0:03}_optimized_with_gps.json".format(fidx), road)


reading: 0
[0.01478732 0.09236399 0.        ]
[ 0.35838396  0.4763409  -2.22160861]
[-0.00779817  0.02295277  0.09034451]
reading: 1
[-0.38402998  3.8184701   0.        ]
[  6.44468386   3.7207631  -24.05091969]
[-1.07567568  0.3601967   3.66624782]
reading: 2
[-1.13362137  8.73394407  0.        ]
[  4.21876379   4.16248613 -26.93533616]
[-1.64190782  0.94126474  8.60145545]
reading: 3
[-0.87704139 11.72946688  0.        ]
[  8.24834236   1.95200821 -22.84873998]
[-4.0768668   0.33961968 11.02784704]
reading: 4
[-0.47777879 14.22329496  0.        ]
[  9.14915524   2.12308797 -25.95852787]
[-4.77944438  0.77552183 13.38229686]
reading: 5
[-0.61811397 16.96487854  0.        ]
[ 11.73477076   0.34161331 -24.00824599]
[-7.45017871 -0.32603873 15.25049851]
